# SQribe 2.0 by AethiQs

## Audio Transcription Application

SQribe is a Python-based application that provides an interactive interface for transcribing audio files to text, refining transcriptions, and downloading the results. Built using Jupyter widgets, it offers a user-friendly way to handle audio transcription tasks.

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
import requests
import openai

# Custom styling
display(HTML("
<style>
.custom-button {
    background-color: #4CAF50;
    border: none;
    color: white;
    padding: 15px 32px;
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 16px;
    margin: 4px 2px;
    cursor: pointer;
}
.transcription-box {
    border: 2px solid #ddd;
    padding: 10px;
    margin: 10px 0;
    border-radius: 5px;
}
</style>
"))

# Create widgets
file_upload = widgets.FileUpload(accept='.mp3,.wav,.m4a', description='Select Audio')
language_select = widgets.Dropdown(
    options=['English', 'Spanish', 'French', 'German', 'Italian'],
    value='English',
    description='Language:'
)
transcribe_button = widgets.Button(description='Transcribe')
transcribe_button.add_class('custom-button')
progress = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:'
)
output = widgets.Output(layout={'border': '1px solid black'})

# Layout
controls = widgets.VBox([
    file_upload,
    language_select,
    transcribe_button,
    progress,
    output
])

# Event handlers
def transcribe_audio(file_path):
    # Default transcription function using AssemblyAI
    headers = {
        'authorization': 'your-assemblyai-api-key',
        'language_code': language_select.value
    }
    with open(file_path, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})
    if response.status_code == 200:
        upload_url = response.json()['upload_url']
        response = requests.post('https://api.assemblyai.com/v2/transcript', json={'audio_url': upload_url}, headers=headers)
        if response.status_code == 200:
            transcript_id = response.json()['id']
            while True:
                response = requests.get(f'https://api.assemblyai.com/v2/transcript/{transcript_id}', headers=headers)
                if response.status_code == 200:
                    if response.json()['status'] == 'completed':
                        return response.json()['text']
                    elif response.json()['status'] == 'failed':
                        return 'Transcription failed'
                else:
                    return 'Error fetching transcription result'
        else:
            return 'Error submitting transcription request'
    else:
        return 'Error uploading file'

def on_transcribe_click(b):
    with output:
        output.clear_output()
        if not file_upload.value:
            print('Please upload a file first')
            return
        
        progress.value = 0
        print('Starting transcription...')
        
        # File upload simulation
        progress.value = 30
        
        # Transcription
        audio_file = next(iter(file_upload.value.values()))
        file_path = f'/tmp/{audio_file['name']}'
        with open(file_path, 'wb') as f:
            f.write(audio_file['content'])
        
        transcript = transcribe_audio(file_path)
        progress.value = 100
        print('Transcription completed!')
        print(transcript)

transcribe_button.on_click(on_transcribe_click)

# Display the interface
display(controls)

# Add keyboard shortcut for transcription
js_code = """
document.addEventListener('keydown', function(event) {
    if (event.ctrlKey && event.key === 't') {
        // Trigger transcription
        document.querySelector('#transcribe-button').click();
    }
});
"""
display(Javascript(js_code))